In [19]:
!pip3 install pulp


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:
import pulp as pl

def compute_optimal_vertex_cover(n, edge_list):
    prob = pl.LpProblem('vert_cover', pl.LpMinimize)
    decision_vars = [pl.LpVariable(f'w_{i}', cat='Binary') for i in range(1, n+1)]
    prob += pl.lpSum(decision_vars) 
    
    # go through each edge in the list
    for (i, j) in edge_list: 
        assert 1 <= i <= n 
        assert 1 <= j <= n
        prob += decision_vars[i-1] + decision_vars[j-1] >= 1
    stat = prob.solve(pl.PULP_CBC_CMD(msg=False)) # solve the problem but suppress output
    assert stat == pl.LpStatusOptimal, 'Problem does not have optimal status -- something went wrong -- this should not happen for this problem'
    vert_cover = [i+1 for i in range(n) if decision_vars[i].varValue > 0 ]
    print(vert_cover)
    return len(vert_cover)

In [21]:
def compute_lp_relaxation_vertex_cover(n, edge_list):
    prob = pl.LpProblem('vert_cover', pl.LpMinimize)
    decision_vars = [pl.LpVariable(f'z_{i}', lowBound=0.0, upBound=1.0, cat='Continuous') for i in range(1, n+1)]
    prob += pl.lpSum(decision_vars)
    
    # go through each edge in the list
    for (i, j) in edge_list: 
        assert 1 <= i <= n 
        assert 1 <= j <= n
        prob += decision_vars[i-1] + decision_vars[j-1] >= 1
    stat = prob.solve(pl.PULP_CBC_CMD(msg=False)) # solve the problem but suppress the output
    assert stat == pl.LpStatusOptimal, 'Problem does not have optimal status -- something went wrong -- this should not happen for this problem'
    vert_cover = [x.varValue for x in decision_vars]
    print(vert_cover)
    return sum(vert_cover)

In [22]:
edges = [(1, 2), (1, 3), (1, 4), 
 (2, 3), (2, 6), 
 (3, 4), (3, 5), (3, 6), 
 (4, 5), (4,7), 
 (5, 6), (5, 7), 
 (6, 7)]

optimal = compute_optimal_vertex_cover(7, edges)
lp_relax = compute_lp_relaxation_vertex_cover(7, edges)
print(optimal)
print(lp_relax)

[2, 3, 4, 6, 7]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
5
3.5
